In [1]:
import gensim
from gensim.models import KeyedVectors
import numpy as np
import torch
from torch import nn, autograd, optim
import torch.nn.functional as F
from sru import SRU, SRUCell
import time
import math
from tqdm import tqdm
import collections
import random

In [2]:
fvec = KeyedVectors.load_word2vec_format('vec_100d.txt', binary=False)
word_vec = fvec.vectors
vocab = ['<PAD>', '<BOS>', '<EOS>', '<UNK>']
vocab.extend(list(fvec.vocab.keys()))
word_vec = np.concatenate((np.array([[0]*word_vec.shape[1]] * 4), word_vec))
word_vec = torch.tensor(word_vec).float()

In [3]:
word_to_idx = {ch: i for i, ch in enumerate(vocab)}
idx_to_word = {i: ch for i, ch in enumerate(vocab)}

In [4]:
essays = []
topics = []
with open('composition.txt', 'r') as f:
    for line in f:
        essay, topic = line.replace('\n', '').split(' </d> ')
        essays.append(essay.split(' '))
        topics.append(topic.split(' '))

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [5]:
corpus_indice = list(map(lambda x: [word_to_idx[w] for w in x], essays[:80000]))
topics_indice = list(map(lambda x: [word_to_idx[w] for w in x], topics[:80000]))
corpus_test = list(map(lambda x: [word_to_idx[w] for w in x], essays[80000:80800]))
topics_test = list(map(lambda x: [word_to_idx[w] for w in x], topics[80000:80800]))

In [6]:
length = list(map(lambda x: len(x), corpus_indice))

In [7]:
def tav_data_iterator(corpus_indice, topics_indice, batch_size, num_steps):
    epoch_size = len(corpus_indice) // batch_size
    for i in range(epoch_size):
        raw_data = corpus_indice[i*batch_size: (i+1)*batch_size]
        key_words = topics_indice[i*batch_size: (i+1)*batch_size]
        data = np.zeros((len(raw_data), num_steps+1), dtype=np.int64)
        for i in range(batch_size):
            doc = raw_data[i]
            tmp = [1]
            tmp.extend(doc)
            tmp.extend([2])
            tmp = np.array(tmp, dtype=np.int64)
            _size = tmp.shape[0]
            data[i][:_size] = tmp
        key_words = np.array(key_words, dtype=np.int64)
        x = data[:, 0:num_steps]
        y = data[:, 1:]
        mask = np.float32(x != 0)
        x = torch.tensor(x)
        y = torch.tensor(y)
        mask = torch.tensor(mask)
        key_words = torch.tensor(key_words)
        yield(x, y, mask, key_words)

In [8]:
class TATLSTM(nn.Module):
    def __init__(self, hidden_dim, embed_dim, num_layers, weight,
                 num_labels, bidirectional, dropout=0.5, **kwargs):
        super(TATLSTM, self).__init__(**kwargs)
        self.hidden_dim = hidden_dim
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.num_labels = num_labels
        self.bidirectional = bidirectional
        if num_layers <= 1:
            self.dropout = 0
        else:
            self.dropout = dropout
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.embedding.weight.requires_grad = False
        self.rnn = nn.GRU(input_size=self.embed_dim, hidden_size=self.hidden_dim,
                          num_layers=self.num_layers, bidirectional=self.bidirectional,
                          dropout=self.dropout).to(device)
        if self.bidirectional:
            self.decoder = nn.Linear(
                hidden_dim * 2 + self.embed_dim, 1000).to(device)
        else:
            self.decoder = nn.Linear(
                hidden_dim + self.embed_dim, 1000).to(device)
#         self.attn = nn.Linear(self.embed_dim * 5, self.embed_dim)
        self.attn = nn.Linear(5, 1, bias=False)
        self.attn.weight.data.fill_(1)
        

    def forward(self, inputs, topics, hidden=None):
        embeddings = self.embedding(inputs).to(device)
        topics_embed = self.embedding(topics)
        topics_attn = torch.zeros(
            topics_embed.shape[0], 1, self.embed_dim)
        for i in range(len(topics_embed)):
            topics_attn[i] = self.attn(topics_embed[i].t()).t()
#         topics_attn = self.attn(topics_embed.reshape((topics_embed.shape[0], -1)))
#         topics_attn.unsqueeze_(-1)
        topics_attn = topics_attn.permute([0, 2, 1])
        self.rnn.flatten_parameters()
        if hidden is None:
            states, hidden = self.rnn(embeddings.permute([1, 0, 2]))
        else:
            states, hidden = self.rnn(
                embeddings.permute([1, 0, 2]), hidden)
        topics_attn = topics_attn.expand(
            topics_attn.shape[0], topics_attn.shape[1], states.shape[0])
        topics_attn = topics_attn.permute([2, 0, 1]).to(device)
        states_with_topic = torch.cat([states, topics_attn], dim=2)
        outputs = self.decoder(states_with_topic.reshape(
            (-1, states_with_topic.shape[-1])))
        return(outputs, hidden)

    def init_hidden(self, num_layers, batch_size, hidden_dim, **kwargs):
        hidden = torch.zeros(num_layers, batch_size, hidden_dim)
        return hidden

In [9]:
def predict_rnn_ada(topics, num_chars, model, idx_to_word, word_to_idx):
    output = [1]
    topics = [word_to_idx[x] for x in topics]
    topics = torch.tensor(topics)
    topics = topics.reshape((1, topics.shape[0]))
    hidden = torch.zeros(num_layers, 1, hidden_dim)
    if use_gpu:
        hidden = hidden.to(device)
        adaptive_softmax.to(device)
#         topics = topics.to(device)
    for t in range(num_chars):
        X = torch.tensor(output[-1]).reshape((1, 1))
#         X = torch.tensor(output).reshape((1, len(output)))
#         if use_gpu:
#             X = X.to(device)
        pred, hidden = model(X, topics, hidden)
        pred = adaptive_softmax.predict(pred)
        if pred[-1] == 2:
            break
        else:
            output.append(int(pred[-1]))
#             output.append(int(pred.argmax(dim=1)[-1]))
    return(''.join([idx_to_word[i] for i in output[1:]]), output[1:])

In [10]:
def bleu(pred_tokens, label_tokens, k):
    len_pred, len_label = len(pred_tokens), len(label_tokens)
    score = math.exp(min(0, 1 - len_label / len_pred))
    for n in range(1, k+1):
        num_matches = 0
        for i in range(len_pred - n + 1):
            if ' '.join(pred_tokens[i: i + n]) in ' '.join(label_tokens):
                num_matches += 1
        score *= math.pow(num_matches / (len_pred - n + 1), math.pow(0.5, n))
    return score

In [11]:
embedding_dim = 100
hidden_dim = 256
lr = 1e2
momentum = 0.1
num_epoch = 300
use_gpu = True
num_layers = 1
bidirectional = False
batch_size = 128
verbose = 5
vocab_size = len(vocab)
device = torch.device('cuda:0')
loss_function = nn.CrossEntropyLoss()
adaptive_softmax = nn.AdaptiveLogSoftmaxWithLoss(
    1000, len(vocab), cutoffs=[round(vocab_size / 20), 4*round(vocab_size / 20)])

In [12]:
model = TATLSTM(hidden_dim=hidden_dim, embed_dim=embedding_dim, num_layers=num_layers,
                num_labels=len(vocab), weight=word_vec, bidirectional=bidirectional)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
# if use_gpu:
#     model = nn.DataParallel(model)
#     model.to(device)
# optimizer = hvd.DistributedOptimizer(optimizer, named_parameters=model.named_parameters)
# hvd.broadcast_parameters(model.state_dict(), root_rank=0)

In [13]:
predict_rnn_ada(['人们', '看看', '展示', '美丽', '夏天'], 100, model, idx_to_word, word_to_idx)[0]

'蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏蕴藏'

In [14]:
since = time.time()
for epoch in range(num_epoch):
    start = time.time()
    num, total_loss = 0, 0
#     if epoch == 5000:
#         optimizer.param_groups[0]['lr'] = lr * 0.1
    data = tav_data_iterator(
        corpus_indice, topics_indice, batch_size, max(length) + 1)
#     hidden = model.module.init_hidden(num_layers, batch_size, hidden_dim)
    weight = torch.ones(len(vocab))
    weight[0] = 0
    for X, Y, mask, topics in tqdm(data):
        num += 1
#         hidden.detach_()
        if use_gpu:
            #             X = X.to(device)
            Y = Y.to(device)
            mask = mask.to(device)
#             topics = topics.to(device)
#             hidden = hidden.to(device)
            weight = weight.to(device)
        optimizer.zero_grad()
#         output, hidden = model(X, topics, hidden)
        output, hidden = model(X, topics)
        hidden.detach_()
#         l = F.cross_entropy(output, Y.t().reshape((-1,)), weight)
        l, _ = adaptive_softmax(output, Y.t().reshape((-1,)))
        loss = -l.reshape((-1, batch_size)).t() * mask
        loss = loss.sum(dim=1) / mask.sum(dim=1)
        loss = loss.mean()
        loss.backward()
        norm = nn.utils.clip_grad_norm_(model.parameters(), 1e-2)
        optimizer.step()
        total_loss += loss.item()
        params = model.state_dict()
        params['attn.weight'].clamp_(1)
    end = time.time()
    s = end - since
    h = math.floor(s / 3600)
    m = s - h * 3600
    m = math.floor(m / 60)
    s -= (m * 60 + h * 3600)
    if((epoch + 1) % verbose == 0) or (epoch == (num_epoch - 1)):
        bleu_score = 0
#         for i in range(len(corpus_test)):
#             doc = corpus_test[i]
#             _, pred = predict_rnn([idx_to_word[x] for x in topics_test[0]],
#                                   100, model, device, idx_to_word, word_to_idx)
#             bleu_score += bleu([idx_to_word[int(x)] for x in pred],
#                                [idx_to_word[x] for x in doc if x not in [0, 2]], k=2)
        print('epoch %d/%d, loss %.4f, norm %.4f, predict bleu: %.4f, time %.3fs, since %dh %dm %ds'
              % (epoch + 1, num_epoch, total_loss / num, norm, bleu_score / 800, end - start, h, m, s))
        print(predict_rnn_ada(['人们', '看看', '展示', '美丽', '夏天'],
                              100, model, idx_to_word, word_to_idx)[0])

625it [01:47,  5.78it/s]
625it [01:47,  5.93it/s]
625it [01:46,  6.06it/s]
625it [01:45,  5.88it/s]
625it [01:46,  5.92it/s]
0it [00:00, ?it/s]

epoch 5/300, loss 6.3886, norm 0.2059, predict bleu: 0.0000, time 106.220s, since 0h 8m 53s
我的喜欢是一个美丽的人，它的是是美丽的。它的是是美丽的。它的是是美丽的。它的是是美丽的。


625it [01:46,  6.01it/s]
625it [01:45,  5.95it/s]
625it [01:45,  5.67it/s]
625it [01:45,  5.41it/s]
625it [01:45,  6.03it/s]
0it [00:00, ?it/s]

epoch 10/300, loss 6.1049, norm 0.1620, predict bleu: 0.0000, time 105.912s, since 0h 17m 42s
我的喜欢有了，它的喜欢是美丽的。它的是很美丽的。它的是很美丽的。它的是很美丽的。它的是很美丽的。


625it [01:45,  5.87it/s]
625it [01:45,  5.72it/s]
625it [01:45,  6.07it/s]
625it [01:46,  5.97it/s]
625it [01:46,  5.83it/s]
0it [00:00, ?it/s]

...

epoch 290/300, loss 5.6809, norm 0.2413, predict bleu: 0.0000, time 105.219s, since 8h 30m 54s
我的家乡在这里，它的美丽的。它像一个美丽的大，像一把把它的大眼睛，让人们人们都喜欢它。它的生命是多么美丽，它是多么美丽的。


625it [01:45,  5.97it/s]
625it [01:45,  5.95it/s]
625it [01:45,  5.97it/s]
625it [01:45,  5.79it/s]
625it [01:44,  5.94it/s]
0it [00:00, ?it/s]

epoch 295/300, loss 5.6810, norm 0.2288, predict bleu: 0.0000, time 104.904s, since 8h 39m 41s
我的家乡是一个美丽的地方，它的人们有一个地方，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，它很美丽，


625it [01:45,  5.94it/s]
625it [01:45,  5.74it/s]
625it [01:45,  6.17it/s]
625it [01:43,  6.10it/s]
625it [01:42,  6.21it/s]


epoch 300/300, loss 5.6804, norm 0.2239, predict bleu: 0.0000, time 102.894s, since 8h 48m 24s
我的家乡在我的身边，它的美丽很美丽，它很美丽，人们都是它的生日。它的美丽的美丽，它的美丽，它的美丽，它的美丽，它的美丽，它的美丽，它的美丽，它的美丽，它的美丽，它的美丽，它的美丽，让人们在这美丽的世界里。


In [15]:
bleu_score = 0
for i in range(len(corpus_test)):
    doc = corpus_test[i]
    _, pred = predict_rnn_ada([idx_to_word[x] for x in topics_test[0]],
                              100, model, idx_to_word, word_to_idx)
    bleu_score += bleu([idx_to_word[int(x)] for x in pred],
                       [idx_to_word[x] for x in doc if x not in [0, 2]], k=2)

In [16]:
predict_rnn_ada(['小朋友', '学习', '英语', '分数', '<PAD>'], 100, model, idx_to_word, word_to_idx)[0]

'我的爸爸很高兴，我很喜欢他。他很高兴，我很喜欢看。他很高兴，我很喜欢他。他说：“你是个好学生，我要学习学习。”'

In [17]:
print(''.join([idx_to_word[x] for x in pred]))

我的妈妈是个“小金鱼”的“小”。妈妈说：“妈妈，我是一个好的。”妈妈说：“你是个好的。”妈妈说：“你是谁的吗？”


In [18]:
print(' '.join([idx_to_word[x] for x in topics_test[len(topics_test) - 1]]))

音乐 节目 兴奋 响起 小手
